Import Libraries

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from scipy.stats import kruskal
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px
import seaborn as sns
matplotlib.use('TkAgg')

import openpyxl

from supervised.automl import AutoML # mljar-supervised

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


Load Dataset

In [37]:
# Load the CSV dataset
df = pd.read_csv('GiveMeSomeCredit/cs-training.csv')
df2 = pd.read_csv('GiveMeSomeCredit/cs-test.csv')

Cleaning NA Values

In [38]:
print(len(df))

# Remove missing values in the "MonthlyIncome" column
df = df.dropna(subset=['MonthlyIncome'])

print(len(df))

# Replace missing values in the "NumberOfDependents" column with 0
df['NumberOfDependents'].fillna(0, inplace=True)

# Remove rows with <500 value in the "MonthlyIncome" column
df = df[df['MonthlyIncome'] >= 500]

print(len(df))

# Remove rows with >2 value in the "DebtRatio" columns
df = df[df['DebtRatio'] <= 2]

print(len(df))

# # Display basic statistics of the numerical columns
# print(df)

150000
120269
117576
116550


In [39]:
print(len(df2))

# Remove missing values in the "MonthlyIncome" column
df2 = df2.dropna(subset=['MonthlyIncome'])

print(len(df2))

# Replace missing values in the "NumberOfDependents" column with 0
df2['NumberOfDependents'].fillna(0, inplace=True)

# Remove rows with <500 value in the "MonthlyIncome" column
df2 = df2[df2['MonthlyIncome'] >= 500]

print(len(df2))

# Remove rows with >2 value in the "DebtRatio" columns
df2 = df2[df2['DebtRatio'] <= 2]

print(len(df2))

# # Display basic statistics of the numerical columns
# print(df)

101503
81400
79670
79050


Heatmap

In [40]:
# # Create the correlation heatmap
# plt.figure(figsize=(10, 10))
# sns.heatmap(df.corr(), annot=True)

# # Save the heatmap as an image file (e.g., PNG)
# plt.savefig("correlation_heatmap.png")

# # Display the heatmap (optional)
# plt.show()

In [41]:
# # Create the correlation heatmap
# plt.figure(figsize=(10, 10))
# sns.heatmap(df2.corr(), annot=True)

# # Save the heatmap as an image file (e.g., PNG)
# plt.savefig("correlation_heatmap.png")

# # Display the heatmap (optional)
# plt.show()

In [42]:
# Define the columns of interest
columns_of_interest = [
    'RevolvingUtilizationOfUnsecuredLines',
    'age',
    'NumberOfTime30-59DaysPastDueNotWorse',
    'DebtRatio',
    'MonthlyIncome',
    'NumberOfOpenCreditLinesAndLoans',
    'NumberOfTimes90DaysLate',
    'NumberRealEstateLoansOrLines',
    'NumberOfTime60-89DaysPastDueNotWorse',
    'NumberOfDependents'
]

Remove outliers for MI & DR

In [43]:
# import pandas as pd

# def detect_outliers(df, columns_of_interest):
#     outlier_indices = {col: [] for col in columns_of_interest}
#     remaining_values = {col: [] for col in columns_of_interest}

#     for column_of_interest in columns_of_interest:
#         loop_times = 0  # Initialize the loop counter

#         while True:
#             # Calculate the IQR (Interquartile Range) for the column
#             Q1 = df[column_of_interest].quantile(0.25)
#             Q3 = df[column_of_interest].quantile(0.75)
#             IQR = Q3 - Q1

#             # Define the lower and upper bounds to identify outliers
#             lower_bound = Q1 - 1.5 * IQR
#             upper_bound = Q3 + 1.5 * IQR

#             # Find the indices of outliers
#             outliers_indices = df[(df[column_of_interest] < lower_bound) | (df[column_of_interest] > upper_bound)].index

#             # Add the indices of outliers to the dictionary
#             outlier_indices[column_of_interest].extend(outliers_indices)

#             # Search for new outliers in the DataFrame without the current outliers
#             df_without_outliers = df.drop(outliers_indices)
#             new_outliers_indices = df_without_outliers[(df_without_outliers[column_of_interest] < lower_bound) | (df_without_outliers[column_of_interest] > upper_bound)].index

#             # Break the loop if no new outliers are found
#             if len(new_outliers_indices) == 0:
#                 break

#         # Find the remaining (non-outlier) values for the current column
#         remaining_values[column_of_interest] = df[column_of_interest][~df.index.isin(outlier_indices[column_of_interest])].tolist()

#         # Display the number of times outliers were found for the current column
#         print(f"Number of times {column_of_interest} outliers were found:", loop_times)

#     # Calculate the sum of outlier indices in the dictionary
#     total_outliers = sum(len(indices) for indices in outlier_indices.values())

#     # Print the total number of outliers
#     print("Total number of outliers:", total_outliers)

#     # Print the values at outlier indices in ascending order
#     for column, indices in outlier_indices.items():
#         values_at_indices = df.loc[indices, column].tolist()
#         values_at_indices.sort()
#         print(f"Values of {column} at outlier indices (ascending order): {values_at_indices}")

#     # Return the dictionaries containing outlier indices and remaining values
#     return outlier_indices, remaining_values

# # Define the list of columns of interest
# columns_of_interest = ['MonthlyIncome', 'DebtRatio']

# # Call the detect_outliers function
# outlier_indices, remaining_values = detect_outliers(df, columns_of_interest)

# # Flatten the list of outlier indices
# all_outlier_indices = [index for indices in outlier_indices.values() for index in indices]

# # Remove rows with outliers from the DataFrame
# df = df.drop(all_outlier_indices)

# print(len(df))

Z-Score

In [44]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Apply z-score standardization to the selected columns
df[columns_of_interest] = scaler.fit_transform(df[columns_of_interest])

print(df)

        SeriousDlqin2yrs  RevolvingUtilizationOfUnsecuredLines       age  \
0                      1                             -0.019974 -0.443053   
1                      0                             -0.019238 -0.791205   
2                      0                             -0.020389 -0.930466   
3                      0                             -0.022023 -1.487509   
4                      0                             -0.019431 -0.164531   
...                  ...                                   ...       ...   
149994                 0                             -0.021438 -0.094901   
149995                 0                             -0.022766  1.576229   
149996                 0                             -0.021769 -0.512684   
149998                 0                             -0.022923 -1.487509   
149999                 0                             -0.019650  0.879925   

        NumberOfTime30-59DaysPastDueNotWorse  DebtRatio  MonthlyIncome  \
0            

In [45]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Apply z-score standardization to the selected columns
df2[columns_of_interest] = scaler.fit_transform(df2[columns_of_interest])

print(df2)

        SeriousDlqin2yrs  RevolvingUtilizationOfUnsecuredLines       age  \
0                    NaN                             -0.022389 -0.587984   
1                    NaN                             -0.024631  0.388785   
2                    NaN                             -0.026862  0.528324   
3                    NaN                             -0.025603 -0.936830   
4                    NaN                             -0.021781 -1.704292   
...                  ...                                   ...       ...   
101495               NaN                             -0.023274 -1.146138   
101497               NaN                             -0.026852  1.086478   
101498               NaN                             -0.025591 -1.913599   
101499               NaN                             -0.022194 -1.076368   
101502               NaN                             -0.024745 -1.564753   

        NumberOfTime30-59DaysPastDueNotWorse  DebtRatio  MonthlyIncome  \
0            

Second Test Data

ML Model LogisticRegression

In [46]:
# # Define the features (X) and the target variable (y)
# X = df[columns_of_interest]  # You can select the columns you want to use as features
# y = df['SeriousDlqin2yrs']   # The target variable you want to predict

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize the Logistic Regression model
# model = LogisticRegression()

# # Fit the model on the training data
# model.fit(X_train, y_train)

# # Make predictions on the test data
# y_pred = model.predict(X_test)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# conf_matrix = confusion_matrix(y_test, y_pred)
# class_report = classification_report(y_test, y_pred)

# # Print the evaluation metrics
# print(f"Accuracy: {accuracy:.2f}")
# print(f"Confusion Matrix:\n{conf_matrix}")
# print(f"Classification Report:\n{class_report}")

In [47]:
# # Load training data from one file
# train_df = df

# # Load testing data from another file
# test_df = df2

# # Define the features (X) and the target variable (y) for training data
# X_train = train_df[columns_of_interest]  # Use the same columns_of_interest as before
# y_train = train_df['SeriousDlqin2yrs']

# # Define the features (X) and the target variable (y) for testing data
# X_test = test_df[columns_of_interest]  # Use the same columns_of_interest as before
# y_test = test_df['SeriousDlqin2yrs']

# # Initialize the Logistic Regression model
# model = LogisticRegression()

# # Fit the model on the training data
# model.fit(X_train, y_train)

# # Make predictions on the test data
# y_pred = model.predict(X_test)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# conf_matrix = confusion_matrix(y_test, y_pred)
# class_report = classification_report(y_test, y_pred)

# # Print the evaluation metrics
# print(f"Accuracy: {accuracy:.2f}")
# print(f"Confusion Matrix:\n{conf_matrix}")
# print(f"Classification Report:\n{class_report}")

In [48]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression

# # Assuming you have already loaded your dataset into a DataFrame df
# # Define the features (X) and the target variable (y)
# X = df[columns_of_interest]  # You can select the columns you want to use as features
# y = df['SeriousDlqin2yrs']   # The target variable you want to predict

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize the Logistic Regression model
# model = LogisticRegression()

# # Fit the model on the training data
# model.fit(X_train, y_train)

# # Use predict_proba to get predicted probabilities
# y_probabilities = model.predict_proba(X_test)

# # y_probabilities will be an array where each row contains the probability of class 0 and class 1
# # For example, y_probabilities[0] will contain [probability_class_0, probability_class_1] for the first test sample

# # Create a DataFrame with the predicted probabilities
# probabilities_df = pd.DataFrame(y_probabilities, columns=['Probability_Class_0', 'Probability_Class_1'])

# # Now you have a DataFrame with columns for each class's probability
# # probabilities_df will have the same number of rows as X_test
# print(probabilities_df)

# # Count rows where Probability_Class_1 is higher than Probability_Class_0
# count_higher_probability = (probabilities_df['Probability_Class_1'] > probabilities_df['Probability_Class_0']).sum()

# print(f"Number of rows where Probability_Class_1 is higher than Probability_Class_0: {count_higher_probability}")

In [49]:
# # Assuming you have already loaded your training dataset into a DataFrame df
# # Define the features (X) and the target variable (y) for training
# X_train = df[columns_of_interest]  # You can select the columns you want to use as features
# y_train = df['SeriousDlqin2yrs']   # The target variable you want to predict

# # Initialize the Logistic Regression model
# model = LogisticRegression()

# # Fit the model on the training data
# model.fit(X_train, y_train)

# # Assuming you have already loaded your testing dataset into a separate DataFrame df2
# # Define the features (X) for testing (df2)
# X_test = df2[columns_of_interest]  # Use the same columns_of_interest as for training

# # Use predict_proba to get predicted probabilities
# y_probabilities = model.predict_proba(X_test)

# # y_probabilities will be an array where each row contains the probability of class 0 and class 1
# # For example, y_probabilities[0] will contain [probability_class_0, probability_class_1] for the first test sample

# # Create a DataFrame with the predicted probabilities
# probabilities_df = pd.DataFrame(y_probabilities, columns=['Probability_Class_0', 'Probability_Class_1'])

# # Now you have a DataFrame with columns for each class's probability
# # probabilities_df will have the same number of rows as X_test
# print(probabilities_df)

# # Count rows where Probability_Class_1 is higher than Probability_Class_0
# count_higher_probability = (probabilities_df['Probability_Class_1'] > probabilities_df['Probability_Class_0']).sum()

# print(f"Number of rows where Probability_Class_1 is higher than Probability_Class_0: {count_higher_probability}")

In [50]:
# Assuming you have already loaded your testing dataset into a separate DataFrame df2
# Define the features (X) for testing (df2)
X_test = df2[columns_of_interest]  # Use the same columns_of_interest as for training

# Use predict_proba to get predicted probabilities
y_probabilities = model.predict_proba(X_test)

# Separate the two probability columns into lists
probability_class_0 = y_probabilities[:, 0]  # Probability of class 0
probability_class_1 = y_probabilities[:, 1]  # Probability of class 1

df2[columns_of_interest] = scaler.inverse_transform(df2[columns_of_interest])

# Add the lists as new columns to df2
df2['Probability_Class_0'] = probability_class_0
df2['Probability_Class_1'] = probability_class_1

# Now df2 has two new columns: 'Probability_Class_0' and 'Probability_Class_1'
# You can print or further work with df2 as needed
# print(df2)

In [51]:
# Specify the path where you want to save the Excel file
excel_file_path = r'C:\Users\user\Desktop\GiveMeSomeCredit\output_file.xlsx'

# Use the to_excel method to save df2 as an Excel file
df2.to_excel(excel_file_path, index=False)

# Print a message to confirm that the file has been saved
print(f'DataFrame df2 has been saved to {excel_file_path}')

DataFrame df2 has been saved to C:\Users\user\Desktop\GiveMeSomeCredit\output_file.xlsx


In [52]:
print(df2)

        SeriousDlqin2yrs  RevolvingUtilizationOfUnsecuredLines   age  \
0                    NaN                              0.885519  43.0   
1                    NaN                              0.463295  57.0   
2                    NaN                              0.043275  59.0   
3                    NaN                              0.280308  38.0   
4                    NaN                              1.000000  27.0   
...                  ...                                   ...   ...   
101495               NaN                              0.718874  35.0   
101497               NaN                              0.045230  67.0   
101498               NaN                              0.282653  24.0   
101499               NaN                              0.922156  36.0   
101502               NaN                              0.441842  29.0   

        NumberOfTime30-59DaysPastDueNotWorse  DebtRatio  MonthlyIncome  \
0                                        0.0   0.177513      